In [1]:
import torch

In [ ]:
# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    #padded_images = # добавить нулей с четырех сторон каждого изображения
    ## Top button
    t=list(input_images.size())
    t[2]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 2)
    input_images=torch.cat((input_images,newrow), 2)
    input_images


    ## Sides
    t=list(input_images.size())
    t[-1]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 3)
    input_images=torch.cat((input_images, newrow), 3)
    padded_images=input_images
    
    padded_images=padded_images.type(torch.float)
    return padded_images

In [ ]:
get_padding2d(input_images)

In [ ]:
torch.sum(tensor,dim)

In [ ]:
a=torch.randint(10, (2, 2))
b=torch.randint(10, (2, 2))

print(a,"\n\n",b,"\n\n", a+b, torch.sum(a+b))

## My Conv2d

In [2]:
import torch
from abc import ABC, abstractmethod


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


# Default kernel tensor

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

kernel_w=kernel.shape[-1]
kernel_h=kernel.shape[-2]
print(kernel_w,kernel_h)

In [ ]:
kernel.shape

In [ ]:
torch.squeeze(kernel,dim=0).shape

In [ ]:
a=torch.randint(10, ( 3, 2, 2))
b=torch.randint(10, ( 3, 2, 2))
print(f"a {a}, \n\n b {b}\n")
print(f"a*b {a*b}")

## My kernel tensor

In [3]:
my_kernel = torch.randint(10, (4, 3, 3, 3)).to(torch.float32)
my_kernel.shape

torch.Size([4, 3, 3, 3])

In [ ]:
my_kernel

## My main class

In [4]:
# Сверточный слой через циклы.

## Ограничение работы моего сверточного слоя только при: 
'''
1. padding = 0
2. Размерность ядра свертки - kernel (высота ядра = ширине ядра) 
   kernel.shape[-1] = kernel.shape[-2] = kernel_height =kernel_height 
3. Нет алгортима backwards
'''

class Conv2dLoop(ABCConv2d):

    def create_empty_out_img(self, input_tensor):
        input_width = input_tensor.shape[-1]
        input_height = input_tensor.shape[-2]
        padding = 0
        
        self.output_height = (input_height + 2 * padding - (self.kernel_size - 1) - 1) // self.stride + 1
        self.output_width = (input_width + 2 * padding - (self.kernel_size - 1) - 1) // self.stride + 1
        
        # Cheking batch size 
        if len(input_tensor.shape) == 3: 
            batch_size = 1
        if len(input_tensor.shape) == 4:
            batch_size = input_tensor.shape[-4]

        out_tesor = torch.zeros([ batch_size, self.out_channels, self.output_height, self.output_width])
        return out_tesor

    def conv_one_sample(self, one_sample, filter, idx_batch, idx_filter):
        h_start = 0
        for _ in range(self.output_height): #h_centr
            w_start = 0
            for _ in range(self.output_width): # w_centr
                # window_sliced - окно которое бежит по вертикали и горизонтали
                window_sliced = one_sample[:,   h_start:h_start+self.kernel_size,   w_start: w_start+self.kernel_size]
                self.out_tensor[ idx_batch, idx_filter, h_start, w_start] = torch.sum( window_sliced * self.kernel[idx_filter])
                w_start+=self.stride
            h_start+=self.stride


    def __call__(self, input_tensor):
        # Вычисление свертки с использованием циклов.
        self.out_tensor=self.create_empty_out_img(input_tensor)

        # Check tensor batch or not and unsqueeze if not
        if len(input_tensor.shape)==4:
            is_batch = True
        elif len(input_tensor.shape)==3:
            is_batch = False
            input_tensor = input_tensor.unsqueeze(0)

        # main loops
        for idx_batch, one_sample in enumerate(input_tensor):   # itteration through batch 
            for idx_filter, filter in enumerate(self.kernel):   # itteration through kernel filters 
                self.conv_one_sample(one_sample, filter, idx_batch, idx_filter)

        # squeeze if not batch
        if is_batch:
            self.out_tensor = torch.squeeze(self.out_tensor,dim=0)
        
        return self.out_tensor

    

In [9]:
img = torch.randint(10, (2, 3, 7, 10)).to(torch.float32)  # .to(torch.float32)
img

tensor([[[[8., 6., 5., 3., 1., 4., 1., 1., 2., 6.],
          [0., 7., 4., 5., 5., 1., 8., 2., 1., 6.],
          [4., 7., 3., 0., 6., 0., 6., 3., 0., 5.],
          [5., 2., 5., 0., 1., 6., 4., 5., 5., 9.],
          [3., 6., 0., 2., 6., 6., 9., 2., 1., 7.],
          [0., 6., 8., 0., 8., 8., 1., 5., 0., 3.],
          [6., 5., 4., 9., 5., 7., 0., 4., 7., 4.]],

         [[7., 1., 0., 4., 7., 7., 5., 0., 6., 3.],
          [5., 6., 0., 6., 3., 1., 2., 8., 1., 7.],
          [9., 6., 7., 3., 4., 7., 3., 1., 4., 6.],
          [9., 4., 8., 1., 3., 9., 8., 3., 5., 1.],
          [6., 1., 4., 7., 5., 1., 1., 5., 1., 0.],
          [0., 3., 9., 3., 1., 4., 2., 2., 7., 7.],
          [9., 2., 0., 6., 9., 4., 4., 2., 5., 2.]],

         [[7., 6., 5., 4., 9., 6., 5., 2., 1., 0.],
          [4., 8., 7., 4., 8., 0., 5., 3., 5., 3.],
          [6., 5., 7., 4., 6., 7., 1., 7., 5., 7.],
          [4., 4., 5., 8., 2., 4., 7., 4., 4., 8.],
          [7., 7., 6., 8., 3., 6., 7., 3., 2., 4.],
        

In [ ]:
len(img.shape)

## Get empty tensor

In [10]:
in_channels = 3
out_channels = 4
kernel_size = 3
stride = 1

my_conv = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
# my_conv.set_kernel(kernel)
my_conv.set_kernel(my_kernel)

In [17]:
kernel_hight = my_kernel.shape[-1]
kernel_widith = my_kernel.shape[-2]


torch.Size([3, 3])

## Call my conv3d

In [11]:
my_conv(img)

tensor([[[[635., 478., 512., 420., 459., 454., 315., 354.],
          [607., 483., 468., 492., 408., 489., 460., 449.],
          [530., 500., 440., 464., 549., 476., 404., 455.],
          [508., 420., 555., 514., 462., 549., 367., 341.],
          [547., 484., 434., 584., 520., 464., 443., 306.]],

         [[629., 561., 585., 525., 647., 454., 419., 383.],
          [672., 588., 556., 482., 478., 600., 467., 541.],
          [685., 593., 500., 628., 556., 516., 493., 448.],
          [534., 600., 592., 508., 583., 610., 502., 456.],
          [650., 538., 501., 628., 569., 556., 410., 422.]],

         [[637., 519., 567., 513., 473., 450., 373., 419.],
          [633., 504., 482., 518., 479., 552., 478., 549.],
          [573., 500., 460., 549., 610., 503., 429., 501.],
          [601., 486., 533., 617., 578., 563., 429., 393.],
          [573., 575., 578., 611., 581., 488., 449., 379.]],

         [[641., 615., 607., 623., 629., 406., 522., 479.],
          [676., 654., 598., 472.,

## Torch conv2d

In [ ]:
# conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
#                                       stride, padding=0, bias=False)

In [12]:
# in_channels = 3
# out_channels = 4
# kernel_size = 3
# stride = 1

in_channels = 3
out_channels = 4
kernel_size = 3
stride = 1


conv2d_torch = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)
conv2d_torch.weight.data = my_kernel # kernel #my_kernel

conv2d_torch(img)

tensor([[[[635., 478., 512., 420., 459., 454., 315., 354.],
          [607., 483., 468., 492., 408., 489., 460., 449.],
          [530., 500., 440., 464., 549., 476., 404., 455.],
          [508., 420., 555., 514., 462., 549., 367., 341.],
          [547., 484., 434., 584., 520., 464., 443., 306.]],

         [[629., 561., 585., 525., 647., 454., 419., 383.],
          [672., 588., 556., 482., 478., 600., 467., 541.],
          [685., 593., 500., 628., 556., 516., 493., 448.],
          [534., 600., 592., 508., 583., 610., 502., 456.],
          [650., 538., 501., 628., 569., 556., 410., 422.]],

         [[637., 519., 567., 513., 473., 450., 373., 419.],
          [633., 504., 482., 518., 479., 552., 478., 549.],
          [573., 500., 460., 549., 610., 503., 429., 501.],
          [601., 486., 533., 617., 578., 563., 429., 393.],
          [573., 575., 578., 611., 581., 488., 449., 379.]],

         [[641., 615., 607., 623., 629., 406., 522., 479.],
          [676., 654., 598., 472.,

In [ ]:
a = torch.randint(10, (2, 3, 7, 7) ).to(torch.float32)


In [ ]:
conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

In [ ]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [ ]:
torch_conv2d = Conv2d(3,4,5,1)
torch_conv2d(a).shape
torch_conv2d(a).shape

In [ ]:
a[0].shape

In [ ]:
## Test input 

In [ ]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

in_channels = kernel.shape[1]
in_channels

In [ ]:
batch_size=2
input_height=4
input_width=4
stride=2


print(batch_size * in_channels * \
                            input_height * input_width)

input_tensor = torch.arange(0, batch_size * in_channels *
                            input_height * input_width,
                            out=torch.FloatTensor()) \
    .reshape(batch_size, in_channels, input_height, input_width)
print(input_tensor.shape)
input_tensor.shape[-3]